In [52]:
# https://python.langchain.com/en/latest/modules/agents/toolkits/examples/gmail.html

# Step 1: Create a json file and download the same in the working folder: https://developers.google.com/gmail/api/quickstart/python#authorize_credentials_for_a_desktop_application

# Step 2: pip install google-api-python-client, google-auth-oauthlib, google-auth-httplib2, beautifulsoup4, langchain

# Step 3: Add Users for testing this app In Authconsent Screen
# Issues resolved: https://stackoverflow.com/questions/65184355/error-403-access-denied-from-google-authentication-web-api-despite-google-acc

# Step 4: Enable it by visiting https://console.developers.google.com/apis/api/gmail.googleapis.com/overview?project=26982114008



from langchain.agents.agent_toolkits import GmailToolkit
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI
toolkit = GmailToolkit() 

# tools = toolkit.get_tools()
# print(tools)

import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
os.environ['OPENAI_API_KEY'] = os.environ["OPENAI_API_KEY"]

from langchain import OpenAI
from langchain.agents import initialize_agent, AgentType

llm = OpenAI(temperature=0)

agent = initialize_agent(
    tools=toolkit.get_tools(),
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
)

context = "charging"
# context = "warranty"
# agent.run("Create a gmail draft for me to editor of a letter from the perspective of a sentient parrot"
#           " who is looking to collaborate on some research with her"
#           " estranged friend, a cat. Under no circumstances may you send the message, however.")

email = agent.run(f"Could you search in my inbox for the latest email on inverter with {context} and provide me the complete mail content with sender email, subject and body in json format?")
# chat_messages=[
#     SystemMessage(content='You are an expert assistant with expertize in reading mails'),
#     HumanMessage(content=f'Please provide the complete email content including the body of the mail and do not add or make up anything and return me a json with sender name, email, subject, body:\n TEXT: {email}')
# ]

# # llm=ChatOpenAI(model_name='gpt-3.5-turbo')
# email_json = llm(chat_messages).content


In [53]:
email

{'id': '18d31516417c464d',
 'threadId': '18d3150da81529af',
 'snippet': 'What is the charging time of the inverter battery?',
 'body': 'What is the charging time of the inverter battery?\r\n',
 'subject': 'inverter battery charging time?',
 'sender': 'AKSHAT PANT <akshat.pant06@gmail.com>'}

In [54]:
json_format = agent.run(f"Convert this string : {email} into a json format with keys as sender_email, subject and body")

In [55]:
json_format

{'sender_email': 'AKSHAT PANT <akshat.pant06@gmail.com>',
 'subject': 'inverter battery charging time?',
 'body': 'What is the charging time of the inverter battery?\r\n'}

In [41]:
product_name = agent.run(f"Identify the Product Name and what is the product category from this context and return a json: {json_format['body']}")

In [42]:
product_name

'The product name is PowerGaurd Elite inverter battery and the product category is inverter battery.'

In [56]:
from PyPDF2 import PdfReader
pdfreader = PdfReader('inv_battery.pdf')

text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        text += content

In [57]:
chat_messages = [
    SystemMessage(content='You are an expert assistant with expertise in reading user manuals of products'),
    HumanMessage(content=f'Please provide an answer to the following question {json_format["body"]} from the:\n TEXT: {text}')
]

llm=ChatOpenAI(model_name='gpt-3.5-turbo')

In [58]:
json_format['body']

'What is the charging time of the inverter battery?\r\n'

In [59]:
query_response = llm(chat_messages).content

In [60]:
query_response

'The charging time of the PowerGuard Elite Inverter Battery is 8-10 hours.'

In [48]:
email

"{'sender': 'AKSHAT PANT <akshat.pant06@gmail.com>', 'subject': 'warranty of inverter battery?', 'body': 'What is the warranty of PowerGaurd Elite inverter battery?\r\n'}"

In [49]:
agent.run(f"Send a gmail reply to {json_format['sender_email']} for {json_format['subject']} with response {query_response}")

'Message sent. Message Id: 18d314c7b27ec689'